## 네이버 금융 개별종목 수집

1. 수집 하고자 하는 페이지의 URL을 알아본다.
1. 파이썬의 작은 브라우저 requests 를 통해 URL에 접근한다.
1. response.status_code 가 200 OK 라면 정상 응답입니다.
1. request의 response 값에서 response.text 만 받아옵니다.
1. html 텍스트를 받아왔다면  BeautifulSoup 로 html 을 해석합니다.
1. soup.select 를 통해 원하는 태그에 접근합니다.
1. 목록을 먼저 받아옵니다.
1. 목록에서 행을 하나씩 가져옵니다.
1. 행을 모아서 데이터프레임으로 만듭니다.
=> 판다스라면 read_html로 table로 된 태그에 대해 위 과정을 코드 한 줄로 해줍니다.



### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 주요 종목
    * 삼성전자 : https://finance.naver.com/item/main.naver?code=005930
    * 현대차 : https://finance.naver.com/item/main.naver?code=005380
    * SK하이닉스 : https://finance.naver.com/item/main.naver?code=000660

## 라이브러리 로드

In [1]:
# 라이브러리 로드

import pandas as pd
import numpy as np


## 수집할 URL 정하기

In [2]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "323410"
# item_name = "카카오뱅크"

item_code = "005930"
item_name = "삼성전자"
page_no = 1

# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"


print(url)

https://finance.naver.com/item/sise_day.naver?code=005930&page=1


## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [3]:
import requests

headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"}
print(headers["user-agent"])


response = requests.get(url, headers=headers)

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36


In [4]:
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

In [5]:
from bs4 import BeautifulSoup as bs

html = bs(response.text, "lxml")

html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="

In [6]:
html.title.string

'네이버 금융'

In [7]:
tables = html.select("table")

## pandas 코드 한 줄로 데이터 수집하기

In [8]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
# table


table = pd.read_html(str(tables), encoding="cp949")
#table = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)

table[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022.05.20,68000.0,500.0,67800.0,68400.0,67700.0,12023540.0
2,2022.05.19,67500.0,600.0,66500.0,67600.0,66500.0,17073727.0
3,2022.05.18,68100.0,500.0,68300.0,68700.0,67600.0,16486319.0
4,2022.05.17,67600.0,1300.0,66600.0,67900.0,66600.0,15680447.0
5,2022.05.16,66300.0,200.0,67100.0,67400.0,66100.0,11937555.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2022.05.13,66500.0,1600.0,65300.0,66700.0,65200.0,14551536.0


In [9]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.


In [10]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
# temp

table[0].dropna()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.05.20,68000.0,500.0,67800.0,68400.0,67700.0,12023540.0
2,2022.05.19,67500.0,600.0,66500.0,67600.0,66500.0,17073727.0
3,2022.05.18,68100.0,500.0,68300.0,68700.0,67600.0,16486319.0
4,2022.05.17,67600.0,1300.0,66600.0,67900.0,66600.0,15680447.0
5,2022.05.16,66300.0,200.0,67100.0,67400.0,66100.0,11937555.0
9,2022.05.13,66500.0,1600.0,65300.0,66700.0,65200.0,14551536.0
10,2022.05.12,64900.0,800.0,65200.0,65500.0,64900.0,16414188.0
11,2022.05.11,65700.0,0.0,65500.0,66300.0,65200.0,12330920.0
12,2022.05.10,65700.0,400.0,65900.0,66300.0,65300.0,17235605.0
13,2022.05.09,66100.0,400.0,66300.0,66900.0,66100.0,11858736.0


## 페이지별 데이터 수집 함수 만들기

In [11]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"
    response = requests.get(url, headers=headers)
    html = bs(response.text, "lxml")
    tables = html.select("table")
    table = pd.read_html(str(tables), encoding="cp949")
    table_fin = table[0].dropna()
    return table_fin
# """
# 일자별 시세를 페이지별로 수집
# """ 

In [12]:
# 함수가 잘 만들어졌는지 확인하기

type(get_day_list(item_code, 3))



pandas.core.frame.DataFrame

## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [13]:
import time
from tqdm import trange
# web page 시작번호
page_no = 1
item_code = 377300
item_name = "카카오페이"
# 데이터를 저장할 빈 변수 선언
item_list = []

# while True :
#     item_list.append(get_day_list(item_code, page_no))
#     page_no += 1
#     if get_day_list(item_code, page_no-1).to_dict() == get_day_list(item_code, page_no).to_dict() : break
    
prev=""    
    
while True:
    print(page_no)

    df_one_page = get_day_list(item_code, page_no)
    #마지막 날짜를 가져옴
    curr = df_one_page.iloc[-1,0]
    #마지막 날짜를 비교했을 때 같으면 while 종료
    if curr == prev :
        break
        

    item_list.append(df_one_page)
    page_no += 1
    prev = curr


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [14]:
item_list

[            날짜       종가     전일비       시가       고가       저가       거래량
 1   2022.05.20  94600.0  6500.0  88500.0  94600.0  88100.0  432843.0
 2   2022.05.19  88100.0  1100.0  86000.0  89000.0  85800.0  310088.0
 3   2022.05.18  89200.0   500.0  89600.0  90900.0  88700.0  240472.0
 4   2022.05.17  88700.0  2900.0  85800.0  88700.0  85400.0  214643.0
 5   2022.05.16  85800.0   200.0  86500.0  88700.0  85700.0  277584.0
 9   2022.05.13  86000.0   100.0  85900.0  86600.0  85000.0  307400.0
 10  2022.05.12  85900.0  5500.0  90000.0  90000.0  85000.0  512917.0
 11  2022.05.11  91400.0   500.0  91000.0  92800.0  89700.0  333197.0
 12  2022.05.10  91900.0  1900.0  91200.0  92000.0  89700.0  514324.0
 13  2022.05.09  93800.0  4000.0  96000.0  96700.0  93400.0  462857.0,
             날짜        종가     전일비        시가        고가        저가       거래량
 1   2022.05.06   97800.0  8700.0  104500.0  105000.0   97100.0  735556.0
 2   2022.05.04  106500.0  2000.0  108000.0  109500.0  106000.0  302514.0
 3   20

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [15]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.

df = pd.concat(item_list)

<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [16]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [17]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
# df["종목코드"]
# df["종목명"]

In [18]:
df["종목코드"] = item_code
df["종목명"] = item_name

df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2022.05.20,94600.0,6500.0,88500.0,94600.0,88100.0,432843.0,377300,카카오페이
2,2022.05.19,88100.0,1100.0,86000.0,89000.0,85800.0,310088.0,377300,카카오페이
3,2022.05.18,89200.0,500.0,89600.0,90900.0,88700.0,240472.0,377300,카카오페이
4,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0,377300,카카오페이
5,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0,377300,카카오페이
...,...,...,...,...,...,...,...,...,...
2,2021.11.09,147000.0,6500.0,153500.0,157500.0,146000.0,892617.0,377300,카카오페이
3,2021.11.08,153500.0,16500.0,168500.0,169500.0,152000.0,1394625.0,377300,카카오페이
4,2021.11.05,170000.0,1000.0,167500.0,179000.0,167500.0,1597937.0,377300,카카오페이
5,2021.11.04,169000.0,24000.0,190000.0,191000.0,166000.0,3487030.0,377300,카카오페이


## 컬럼 순서 변경하기

In [19]:



# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.


In [20]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]
df

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,377300,카카오페이,2022.05.20,94600.0,6500.0,88500.0,94600.0,88100.0,432843.0
2,377300,카카오페이,2022.05.19,88100.0,1100.0,86000.0,89000.0,85800.0,310088.0
3,377300,카카오페이,2022.05.18,89200.0,500.0,89600.0,90900.0,88700.0,240472.0
4,377300,카카오페이,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0
5,377300,카카오페이,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0
...,...,...,...,...,...,...,...,...,...
2,377300,카카오페이,2021.11.09,147000.0,6500.0,153500.0,157500.0,146000.0,892617.0
3,377300,카카오페이,2021.11.08,153500.0,16500.0,168500.0,169500.0,152000.0,1394625.0
4,377300,카카오페이,2021.11.05,170000.0,1000.0,167500.0,179000.0,167500.0,1597937.0
5,377300,카카오페이,2021.11.04,169000.0,24000.0,190000.0,191000.0,166000.0,3487030.0


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [21]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인


df = df.drop_duplicates()

## 기술통계값 구하기

In [22]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종목코드,종가,전일비,시가,고가,저가,거래량
count,136.0,136.000000,136.000000,136.000000,136.000000,136.000000,1.360000e+02
mean,377300.0,146475.735294,4708.823529,146444.852941,150959.558824,142647.794118,5.506609e+05
std,0.0,30895.519264,4746.296343,30490.998841,33446.548576,28887.652025,1.097833e+06
min,377300.0,85800.000000,0.000000,85800.000000,86600.000000,85000.000000,8.119600e+04
25%,377300.0,128500.000000,1500.000000,130000.000000,133000.000000,126500.000000,2.127760e+05
50%,377300.0,141500.000000,3750.000000,141500.000000,145000.000000,138500.000000,3.099210e+05
75%,377300.0,169000.000000,6000.000000,168750.000000,173500.000000,166000.000000,5.037890e+05
max,377300.0,238500.000000,33500.000000,240500.000000,248500.000000,216500.000000,1.179988e+07


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [23]:
# 거래량의 과학적 기수법 읽기


## 최근 날짜 구해서 파일명 만들기

In [24]:
# 날짜 column의 첫 row값 확인
# date

date = df.iloc[0]["날짜"]


In [25]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
# file_name


file_name = f"stock_{item_name}_{item_code}_{date}.csv"
file_name

'stock_카카오페이_377300_2022.05.20.csv'

## 파일로 저장하기

In [26]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.


df.to_csv(file_name, index=False)

In [27]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.

pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,377300,카카오페이,2022.05.20,94600.0,6500.0,88500.0,94600.0,88100.0,432843.0
1,377300,카카오페이,2022.05.19,88100.0,1100.0,86000.0,89000.0,85800.0,310088.0
2,377300,카카오페이,2022.05.18,89200.0,500.0,89600.0,90900.0,88700.0,240472.0
3,377300,카카오페이,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0
4,377300,카카오페이,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0
...,...,...,...,...,...,...,...,...,...
131,377300,카카오페이,2021.11.09,147000.0,6500.0,153500.0,157500.0,146000.0,892617.0
132,377300,카카오페이,2021.11.08,153500.0,16500.0,168500.0,169500.0,152000.0,1394625.0
133,377300,카카오페이,2021.11.05,170000.0,1000.0,167500.0,179000.0,167500.0,1597937.0
134,377300,카카오페이,2021.11.04,169000.0,24000.0,190000.0,191000.0,166000.0,3487030.0


## 전체 과정을 하나의 함수로 만들기

In [28]:
def get_item_list(item_code, item_name):
    item_list = []
    page_no = 1

    prev=""    
    
    while True:
        print(page_no)

        df_one_page = get_day_list(item_code, page_no)
        #마지막 날짜를 가져옴
        curr = df_one_page.iloc[-1,0]
        #마지막 날짜를 비교했을 때 같으면 while 종료
        if curr == prev :
            break


        item_list.append(df_one_page)
        page_no += 1
        prev = curr
        
    df = pd.concat(item_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]
    df = df.drop_duplicates()
    date = df.iloc[0]["날짜"]
    file_name = f"stock_{item_name}_{item_code}_{date}.csv"
    df.to_csv(file_name, index=False)
    
    return df

In [29]:
item_code = "323410"
item_name = "카카오뱅크"



get_item_list(item_code, item_name)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,323410,카카오뱅크,2022.05.20,39950.0,1300.0,38650.0,40100.0,38650.0,844086.0
2,323410,카카오뱅크,2022.05.19,38650.0,750.0,38400.0,38750.0,38000.0,1023492.0
3,323410,카카오뱅크,2022.05.18,39400.0,600.0,40150.0,40300.0,39300.0,726873.0
4,323410,카카오뱅크,2022.05.17,40000.0,1150.0,38450.0,40250.0,38450.0,1152307.0
5,323410,카카오뱅크,2022.05.16,38850.0,400.0,38500.0,39750.0,38450.0,1189121.0
...,...,...,...,...,...,...,...,...,...
12,323410,카카오뱅크,2021.08.12,73800.0,600.0,73600.0,75400.0,72700.0,5878196.0
13,323410,카카오뱅크,2021.08.11,74400.0,3000.0,71200.0,77500.0,68100.0,21592734.0
1,323410,카카오뱅크,2021.08.10,71400.0,7100.0,81800.0,85600.0,68000.0,26495122.0
2,323410,카카오뱅크,2021.08.09,78500.0,8700.0,69800.0,89100.0,69000.0,44833696.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [30]:
# 도움말 보기
# get_item_list?


In [31]:
# 소스코드 보기
# get_item_list??
